# GROMACS Molecular Dynamics Visualization

This notebook creates interactive visualization of the completed GROMACS protein MD simulation.

## Install Dependencies

Install required packages for trajectory visualization:

In [ ]:
# Install required packages
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import MDAnalysis as mda
    import nglview as nv
except ImportError:
    install('MDAnalysis')
    install('nglview')
    import MDAnalysis as mda
    import nglview as nv

import os
import ipywidgets as widgets
from IPython.display import display

## Load Trajectory

Load the GROMACS NPT trajectory for visualization:

In [ ]:
# Load GROMACS trajectory
base_path = os.getcwd()

# Load the final NPT trajectory
u = mda.Universe(
    os.path.join(base_path, "output/npt.tpr"),
    os.path.join(base_path, "output/npt_nopbc.xtc")
)

print(f"Loaded trajectory with {len(u.trajectory)} frames")
print(f"System contains {u.atoms.n_atoms} atoms")
print(f"Protein atoms: {u.select_atoms('protein').n_atoms}")
print(f"Water molecules: {u.select_atoms('resname SOL').n_residues}")

## Create Interactive Visualization

Generate 3D molecular visualization with playback controls:

In [ ]:
# Create interactive 3D visualization
view = nv.show_mdanalysis(u, step=5)  # Show every 5th frame for performance

# Clear default representations
view.clear_representations()

# Add protein cartoon representation
view.add_representation('cartoon', selection='protein', color='secondary_structure')

# Add water as small spheres (limited for performance)
view.add_representation('ball+stick', selection='resname SOL and around 8 protein', radius=0.1)

# Add ions if present
try:
    if u.select_atoms('resname NA+ or resname CL-').n_atoms > 0:
        view.add_representation('spacefill', selection='resname NA+ or resname CL-', radius=0.3)
except:
    pass

# Add playback controls
play = widgets.Play(
    value=0,
    min=0,
    max=view.max_frame,
    step=1,
    interval=100,  # ms between frames
    description="▶️",
    continuous_update=True
)

# Link play widget to view
widgets.jslink((play, 'value'), (view, 'frame'))

# Display controls and view
display(widgets.HBox([play]))
print("Use the play button to animate the molecular dynamics trajectory")
view